# Packet Sniffing and Spoofing 
__sniffing__ refers to eavesdrop on physical network, wired or wireless, and capture the packets transmitted over the network. <br>
__spoofing__ refers to send out packets under a false identity. <br>
The tools commonly used are _Wireshark, netwox, scapy_. The low level details are to be covered in C, and the high level usage are to be covered in Python. 

### Network Packet Transmission on physical layer
Machines are connected to network via Network Interface Card (__NIC__). As data flow in the medium, every NIC on the network will hear all the frames on the wire. When a frame arrives via the medium, it is copied into the memory inside the NIC, which checks the destination MAC address to see if it matches the NIC's MAC address. If it matches, the frame is further copied into a buffer in the kernel, through Direct Memory Access (DMA). The card then interrupts the CPU to inform it about the availability of a new packet, and the CPU copies all the packets from the bnuffer into a queue, making room in hte buffer for more incoming packets. Callback handler functions are used by OS to dispatch the packets to user-space programs. 
#### Promiscuous Mode 
In general, frames not destined for the destination NIC are discarded rather than being passed to CPU for processing. However, for most NIC, _promiscuous mode_ allows NIC to not drop and passes every incoming frame to the kernel. If a sniffer program is running, all these frames can be dispatched to the sniffer program for further processing. 
#### Monitor Mode
Monitor mode for wireless network cards supports sniffing too. When in monitor mode, the cards capture 802.11 frames transmitting on the channel that they are listening to. This means that unlike ethernet, in wireless cards, it is possible to miss information on the same network due to different channels. Most wireless NIC has their monitor mode disabled by the manufacturers. 

#### Lowest Packet Filtering: BSD Packet Filter
BSD Packet Filter (BPF) provides filtering at the lowest level on UNIX machines. It allows a user-space program to attach a filter to a socket, which discards unwanted packets as early as possible. The high-level C function is via the `setsockopt` call: The `code` is supposed to be compiled BPF code
```c
struct sock_fprog bpf = {
    .len = ARRAY_SIZE(code),
    .filter = code, 

};

setsockopt(sock, SOL_SOCKET, SO_ATTACH_FILTER, &bpf, sizeof(bpf)); 
```

### Sniffing 
In general, the task at hand is to capture live data as they flow across a network. The following C program demonstrates a simple UDP connection setup: 
```c
#include <stdio.h>
#include <string.h>
#include <sys/socket.h>
#include <netinet/ip.h>

void main() 
{
    struct sockaddr_in server;
    struct sockaddr_in client;
    int clientlen;
    char buf[1500];
    
    //step 1: create a socket 
    int sock = socket(AF_INET, SOCK_DGRAM, IPPROTO_UDP);
    
    //step 2: provide information about the server, such as bind UDP, IP
    // binding is for all available IP address of the network interface
    memset((char *) &server, 0, sizeof(server)); 
    server.sin_family = AF_INET;
    server.sin_addr.s_addr = htonl(INADDR_ANY); 
    server.sin_port = htons(9090); //listen on UDP port 9090 for incoming packets 
    
    if (bind(sock, (struct sockaddr *) &server, sizeof(server)) < 0)
        error("Error on binding");
    
    //step 3: wait for response from client 
    while (1) {
        bzero(buf, 1500); 
        recvfrom(sock, buf, 1500-1, 0, (struct sockaddr *) &client, &clientlen); 
        printf("%s\n", buf);                
    }
    close(sock);
}

```
The similar UDP server in Python follows the same step: create socket, bind, and listen for connections. 

In [ ]:
import socket 
UDP_IP = "127.0.0.1"
UDP_PORT = 9090
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind((UDP_IP, UDP_PORT))
while True: 
    data, addr = sock.recvfrom(1024)
    print "recieved data: ", data 

### Packet Sniffing using Raw sockets 
since sniffer needs to capture all the packets, it is not plausible to hard code an IP address in the program for receiving packets. Therefore, instead of using a general socket, we use a __raw socket__. 

```c
#include <stdio.h>
#include <string.h>
#include <sys/socket.h>
#include <netinet/ip.h>

void main() 
{
    int PACKET_LEN = 512;
    char buffer[PACKET_LEN]; 
    struct sockaddre saddr;
    struct packet_mreq mr; 
    //step 1: create a RAW socket 
    //protocol is set to be ETH_P_ALL, which means receiving all protocols 
    //if it is ETH_P_IP, only IP packets will be captured. 
    int sock = socket(AF_INET, SOCK_RAW, htons(ETH_P_ALL));
    //step 2: turn on promiscuous mode 
    mr.mr_type = PACKET_MR_PROMISC;
    // PACKET_ADD_MEMBERSHIP, PACKET_MR_PROMISC are NIC specific, need to search before committing. 
    setsocktopt(sock, SOL_PACKET, PACKET_ADD_MEMBERSHIP, &mr, sizeof(mr)); 
    
    //step 3: getting captured packets  
    while (1) {
        int data_size = recvfrom(sock, buffer, PACKET_LEN, 0, 
                                 &saddr, (socklen_t *)sizeof(saddr));
        if (data_size) printf("Got one packet\n"); 
    }
    close(sock);
    return 0; 
}
```
This sniffer has no filtering, and could easily lose packets. A better option is to use the `pcap` API.

#### pcap sniffer
The following program captures all ICMP packets. 
```c
#include <pcap.h>
#include <stdio.h>

void got_packet(u_char *args, const struct pcap_pkthdr *header, const u_char *packet)
{
    printf("Got a packet"); 
}

int main() 
{
    pcap_t *handle;
    char errbuf[PCAP_ERRBUF_SIZE]; 
    struct bpf_program fp;
    char filter_exp[] = "ip proto icmp"; 
    bpf_u_int32 net; 
    
    //step 1: open live pcap session on NIC with name eth3 
    handle = pcap_open_live("eth3", BUFSIZ, 1, 1000, errbuf);
    
    //step 2: compile filter_exp into BPF pseudo-code
    pcap_compile(handle, &fp, filter_exp, 0, net);
    pcap_setfilter(handle, &fp); 
    
    //step 3: capture packets
    pcap_loop(handle, -1, got_packet, NULL); 
    pcap_close(handle);
    return 0;
}

```
pcap filters are be written in formats such as:
* `dst host 10.0.0.25` only captures packets going to 10.0.0.25.
* `src host 10.0.0.25` only captures packets with source 10.0.0.25
* `host 10.0.0.5 and src port 9090`: only captures packets from or going to 10.0.0.5 and with source port 9090.
* `proto tcp`: only capture TCP packets

To compile a pcap program, use gcc to compile with `-lpcap` flag:
`gcc -o sniff sniff.c -lpcap`


### Processing Captured Packets
when `got_packet()` is invoked, the third argument points to the buffer, which is an ethernet frame, with ethernet header placed at the beginning. <br>
In C, to read such a chunk of memory (buffer), we refer to the structure of an ethernet packet defined in RFC and defined it in C as a struct. The data in the struct should be strictly the same as the actual ethernet packet. <br> 
```c
struct ethheader 
{
    u_char ether_dhost[ETHER_ADDR_LEN];  //dst host address 
    u_char ether_shost[ETHER_ADDR_LEN];  //src host address 
    u_short ether_type;                  //type of packet, could be IP, ARP, etc. 
};

void got_packet(u_char *args, const struct pcap_pkthdr *header, const u_char *packet)
{
    struct ethheader *eth = (struct ethheader *) packet;   // we use type casting to read the packet by our struct
    if (ntohs(eth->ether_type) == 0x0800) { .. } // if it is IP 
    ... 
}

```

To further read the IP packet inside the ethernet packet, we need to first move to an offset equivalent to the ethernet header, and then read the buffer using a struct identical to the structure defined in RFC for IP packet. <br> 
![header](./image_files/IP_header.gif)
```c
#include <pcap.h>
#include <stdio.h>
#include <arpa/inet.h>

struct ipheader 
{
      unsigned char ip_ihl:4, //  4 bits, IP header length 
                    ip_ver:4; //  4 bits, IP version 
      unsigned char iph_tos;  //  8 bits, Type of service
      unsigned short int iph_len; // 16 bits, IP packet length
      unsigned short int iph_ident; // 16 bits, identification 
      unsigned short int iph_flag:3,  // fragmentation flags 
                         iph_offset:13; //flag offset 
      unsigned char iph_ttl;   // 8 bits, time to live
      unsigned char iph_protocol; //protocol type 
      unsigned short int iph_chksum; 
      struct in_addr iph_sourceip;   //source IP address 
      struct in_addr iph_destip;     //destination iP address 
};

void got_packet(u_char *args, const struct pcap_pkthdr *header, const u_char *packet) 
{
    struct ethheader *eth = (struct ethheader *) packet;
    if (ntohs(eth->ether_type == 0x8000))
    {
        struct ipheader *ip = (struct ipheader *) (packet + sizeof(struct ethheader));
        printf("From: %s\n", inet_ntoa(ip->iph_sourceip));
        printf("To: %s\n", inet_ntoa(ip->iph_destip)); 
        //determine protocol 
        switch(ip->iph_protocol)
        {
            case IPPROTOCOL_TCP: 
                printf(" protocol: TCP\n");
                return;
        
            case IPPROTOCOL_UDP: 
                printf(" protocol: UDP\n"); 
                return;
        
            case IPPROTOCOL_ICMP:
                printf(" protocol ICMP\n");
                return; 
        
            default:
                printf(" protocol others\n");
                return; 
        }
    }       
}
```
we can further read the packet (such as ICMP packet, TCP packet, or UDP packet) by using the same strategy: using corersponding struct to read the buffer, with specified offset defined by the RFC files. 

### Common Packet header structures: 
1. Layers in general 
![layer](./image_files/layers.jpg)


#### Ethernet Header 
```c
/* This is a name for the 48 bit ethernet address available on many
   systems.  */
struct ether_addr
{
  u_int8_t ether_addr_octet[ETH_ALEN];
} __attribute__ ((__packed__));

/* 10Mb/s ethernet header */
struct ether_header
{
  u_int8_t  ether_dhost[ETH_ALEN];	/* destination eth addr	*/
  u_int8_t  ether_shost[ETH_ALEN];	/* source ether addr	*/
  u_int16_t ether_type;		        /* packet type ID field	*/
} __attribute__ ((__packed__));
```

#### TCP header 
![tcp](./image_files/tcp-header.png)

#### UDP datagram 
![udp](./image_files/udp.jpeg)

In general, the length of IP packet is determined at run time, through `iph_ihl` in the previous header struct definition. Suppose the packet after IP is ICMP: 
```c
int ip_header_len = ip->iph_ihl * 4; 
u_char *icmp = (struct icmpheader *) (packet + sizeof(struct ethheader) + ip_header_len); 
```

### Packet Spoofing 

First, a program to send UDP datagram with provided payload (UDP client) in C:
```c
#include <stdio.h>
#include <string.h> 
#include <sys/socket.h>
#include <netinet/ip.h>

void main()
{
    struct sockaddr_in dest_info; 
    char *data = "UDP message\n"; 
    //step 1: create a network socket
    int sock = socket(AF_INET, SOCK_DGRAM, IPPROTO_UDP);
    
    //step 2: provide information about destination 
    memset((char *) &dest_info, 0, sizeof(dest_info)); 
    dest_info.sin_family = AF_INET;
    dest_info.sin_addr.s_addr = inet_addr("10.11.0.24");
    dest_info.sin_port = htons(9090);
    
    //step 3: send out the packet
    sendto(sock, data, strlen(data), 0, (struct sockaddr *) &dest_info, sizeof(dest_info)); 
    close(sock); 
}

```

Then, client that sends crafted packets with raw sockets: two steps are invovled, (__NOte that only root can generate and send RAW sockets)__. 
* Construct the packet in a buffer
* send the packet out 

step 2 is achievd in C code such as the following: 
```c
//given an IP packet, send it out using raw socket
void send_raw_ip_packet(struct ipheader* ip) 
{
    struct sockaddr_in dest_info; 
    int enable = 1;
    
    //step 1: create a raw network socket
    // IPPROTO_RAW indicates we are supplying the ip header ourselves 
    int sock = socket(AF_INET, SOCK_RAW, IPPROTO_RAW); 
    
    // step 2: set socket option 
    // IPPROTO_RAW indicates the flag IP_HDRINCL, which means we include our own IP header 
    // this line can be ommitted, since IPPROTO_RAW already issued. 
    setsockopt(sock, IPPROTO_IP, IP_HDRINCL, &enable, sizeof(enable)); 
    
    // step3: provide needed information about the destination 
    // IP packet, destination 
    dest_info.sin_family = AF_INET;
    dest_info.sin_addr = ip->ip_destip;
    
    //step 4: send the packet out
    // second argument (ip here) is the pointer to the buffer containing the whole IP packet 
    // third argument is the size of the packet 
    // fourth and fifth are standard socket options for destination 
    sendto(sock, ip, ntohs(ip->lph_len), 0, (struct sockaddr *) &dest_info, sizeof(dest_info)); 
    close(sock); 
}

```

Step 1 is to construct the buffer for our packet. 
#### ICMP Packet Construction 
Suppose we are sending a crafted ICMP Echo request message, with a spoofed source IP address. 
```c
#include <stdio.h>
#include <string.h>
#include <sys/socket.h>
#include <netinet/ip.h>



void send_raw_ip_packet(struct ipheader* ip) 
{
    struct sockaddr_in dest_info; 
    int sock = socket(AF_INET, SOCK_RAW, IPPROTO_RAW); 
    dest_info.sin_family = AF_INET;
    dest_info.sin_addr = ip->ip_destip;
    sendto(sock, ip, ntohs(ip->lph_len), 0, (struct sockaddr *) &dest_info, sizeof(dest_info)); 
    close(sock); 
}


// ICMP header 
struct icmpheader 
{
    unsigned char icmp_type;  // ICMP message type 
    unsigned char icmp_code;  // Error code 
    unsigned short int icmp_chksum; 
    unsigned short int icmp_id;
    unsigned short int icmp_seq; 
};


int main()
{
    char buffer[1500];
    memset(buffer, 0, 1500); 
    
    //Step 1: fill in the ICMP header 
    struct icmpheader *icmp = (struct icmpheader *) (buffer + sizeof(struct ipheader)); 
    icmp->icmp_type = 8;    // 8 is request 
    icmp->icmp_chksum = 0; 
    icmp->icmp_chksum = in_cksum((unsigned short *) icmp, sizeof(struct icmpheader)); 
    
    //step 2: fill in the IP header 
    // use the same ipheader struct as before 
    struct ipheader *ip = (struct ipheader *) buffer;
    ip->iph_ver = 4;
    ip->iph_ihl = 5;
    ip->iph_ttl = 20;
    ip->iph_sourceip.s_addr = inet_addr("1.2.3.4"); 
    ip->iph_destip.s_addr = inet_addr("10.0.2.4");
    ip->iph_protocol = IPPROTO_ICMP;
    ip->iph_len = htons(sizeof(struct ipheader) + sizeof(struct icmpheader)); 
    
    //step3: send spoofed packet 
    send_raw_ip_packet(ip); 
    return 0; 
}
```

#### UDP Packet Creation 
```c
#include <stdio.h>
#include <sys/socket.h>
#include <netinet/ip.h>

struct udpheader 
{
    u_int16_t udp_sport; 
    u_int16_t udp_dport; 
    u_int16_t udp_ulen;
    u_int16_t udp_sum; 
};


void send_raw_ip_packet(struct ipheader* ip) 
{
    struct sockaddr_in dest_info; 
    int sock = socket(AF_INET, SOCK_RAW, IPPROTO_RAW); 
    dest_info.sin_family = AF_INET;
    dest_info.sin_addr = ip->ip_destip;
    sendto(sock, ip, ntohs(ip->lph_len), 0, (struct sockaddr *) &dest_info, sizeof(dest_info)); 
    close(sock); 
}


int main() 
{
    char buffer[1500]; 
    memset(buffer, 0, 1500); 
    struct ipheader *ip = (struct ipheader *) buffer; 
    struct icmpheader *icmp = (struct icmpheader *) (buffer + sizeof(struct ipheader));
    
    // step1: fill in the UDP data field 
    char *data = buffer + sizeof(struct ipheader) + sizeof(struct udpheader); 
    const char *msg = "Hello Server\n"; 
    int data_len = strlen(msg);
    
    strncpy(data, msg, data_len); 
        
    //step 2: fill in UDP header 
    udp->udp_sport = htons(12345);
    udp->udp_dport = htons(9090);
    udp->udp_ulen = htons(sizeof(struct udpheader) + data_len); 
    udp->udp_sum = 0; 
    
    //step 3: fill in IP header 
    ip->iph_ver = 4;
    ip->iph_ihl = 5;
    ip->iph_ttl = 20;
    ip->iph_sourceip.s_addr = inet_addr("1.2.3.4"); 
    ip->iph_destip.s_addr = inet_addr("10.0.2.4");
    ip->iph_protocol = IPPROTO_UDP;    
    ip->iph_len = htons(sizeof(struct ipheader) + sizeof(struct udpheader) + data_len); 
    
    // send packet
    send_raw_ip_packet(ip);
    
    return 0; 
}

```

### Snoofing: Sniffing and Spoofing 

sniff then construct spoofed reply based on the contents of the captured packets.<br> 
The example demonstrates snoofing for UDP packet destined for port 9999. The spoof part is the same as before, here is the function called in the handler of pcaploop. <br>
```c
void spoof_reply(struct ipheader* ip) 
{
    const char buffer[1500]; 
    int ip_header_len = ip->ip_ihl * 4; 
    struct udpheader * udp = (struct udpheader *) ((u_char *) ip + ip_header_len);
    if (ntohs(udp->udp_dport) != 9999) 
        return;
        
    //step 1: make a copy from the original packet 
    memset((char *)buffer, 0, 1500); 
    memcpy((char *)buffer, ip, ntohs(ip->iph_len)); 
    struct ipheader *newip = (struct ipheader *) buffer;
    struct udpheader *newudp = (struct udpheader *) (buffer + ip_header_len); 
    char *data = (char *)newudp + sizeof(struct udpheader); 
    
    //step2: construct the UDP payload, keep track of payload size
    const char *msg = "This is a spoofed reply!\n"; 
    int data_len = strlen(msg); 
    strncpy(data, msg, data_len); 
    
    //step3: construct the UDP header 
    newudp->udp_sport = udp->udp_dport;
    newdup->udp_dport = udp->udp_sport;
    newdup->udp_ulen = htons(sizeof(struct udpheader) + data_len); 
    newdup->udp_sum = 0;
    
    //step4: Construct IP header 
    newip->iph_sourceip = ip->iph_destip;
    newip->iph_destip = ip->iph_sourceip;
    newip->iph_ttl = 50;
    newip->iph_len = htons(sizeof(struct ipheader) + sizeof(struct udpheader) + data_len); 
    
    //step 5: send the packet
    send_raw_ip_packet(newip); 
    
}
```

### Endianness 
`htons` and `ntohs` are used to convert byte endianness: the network uses big endian (or _network byte order_) while hosts normally use little endian format:
* `htons()` converts unsigned short integer from host order to network order
* `htonl()` converts unsigned integer from host order to network order
* `ntohs()` converts unsigned short integer from network to host order
* `ntols()` converts unsigned integer from host order to network order 
In general, all the packets received from the network needs to be processed with either `ntohs` or `ntols`, whereas all the packets sent to the network needs to be processed with either `htons` or `htols`. 

### Checksum Calculation


#### UDP checksum
```c
unsigned short in_cksum(unsigned short *buf, int length) 
{
    unsigned short *w = buf;
    int nleft = length;
    int sum = 0;
    unsigned short temp = 0;
    
    while(nleft > 1) 
    {
        sum += *w++;
        nleft -= 2; 
    }
    
    if (nleft == 1) 
    {
        *(u_char *) (&temp) = *(u_char *) 2; 
        sum += temp; 
    }
    
    sum = (sum >> 16) + (sum & 0xffff); 
    sum += (sum >> 16); 
    return (unsigned short) (~sum); 
}

```

#### TCP checksum 
```c
/* TCP header */
typedef u_int tcp_seq;

struct tcpheader {
	u_short th_sport;	/* source port */
	u_short th_dport;	/* destination port */
	tcp_seq th_seq;		/* sequence number */
	tcp_seq th_ack;		/* acknowledgement number */
	u_char th_offx2;	/* data offset, rsvd */
	#define TH_OFF(th)	(((th)->th_offx2 & 0xf0) >> 4)
    u_char th_flags;
	#define TH_FIN 0x01
	#define TH_SYN 0x02
	#define TH_RST 0x04
	#define TH_PUSH 0x08
	#define TH_ACK 0x10
	#define TH_URG 0x20
	#define TH_ECE 0x40
	#define TH_CWR 0x80
	#define TH_FLAGS (TH_FIN|TH_SYN|TH_RST|TH_ACK|TH_URG|TH_ECE|TH_CWR)
	u_short th_win;		/* window */
	u_short th_sum;		/* checksum */
	u_short th_urp;		/* urgent pointer */
};


//TCP pseudo header
struct pseudo_tcp
{
    unsigned saddr, daddr; 
    unsigned char mbz; 
    unsigned char ptcl;
    unsigned short tcpl;
    struct tcpheader tcp;
    char payload[PACKET_LEN]; 
};

unsigned short calculate_tcp_checksum(struct ipheader *ip)
{
    struct tcpheader *tcp = (struct tcpheader*) ((u_char *) ip + sizeof(struct ipheader)); 
    int tcp_len = ntohs(ip->ip_len) - sizeof(struct ipheader); 
    
    struct pseudo_tcp p_tcp; 
    memset(&p_tcp, 0x0, sizeof(struct pseudo_tcp));
    
    p_tcp.saddr = ip->iph_sourceip.s_addr; 
    p_tcp.daddr = ip->iph_destip.s_addr;
    p_tcp.mbz = 0;
    p_tcp.ptcl = IPPROTO_TCP;
    p_tcp.tcpl = htons(tcp_len);
    memcpy(&p_tcp->tcp, tcp, tcp_len); 
    
    return (unsigned short) in_cksum((unsigned short *) &p_tcp, tcp_len + 12); 
}


```